In [ ]:
'''
to explore:
-additional date extraction
-tuning rf
-add additional columns
-log before fit
-aggregate for customers before fit

'''

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', 500)

In [2]:
train = pd.read_csv('train-flattened.csv',dtype={'date': str, 'fullVisitorId': str, 'sessionId':str, 'visitId': np.int64})

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.read_csv('test-flattened.csv',dtype={'date': str, 'fullVisitorId': str, 'sessionId':str, 'visitId': np.int64})

In [4]:
#lets look at columns
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,1.0,1,1.0,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,1.0,1,1.0,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,1.0,1,1.0,1.0,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo d

In [5]:
#What is shape of df?
print(train.shape)
print(test.shape)

(903653, 55)
(804684, 53)


In [6]:
#what percent of rows are nan?
revenue = train['totals.transactionRevenue']
revenue.isna().sum()/len(revenue)

0.9872572768529513

In [7]:
#what percent of visitors have no revenue when visits combined?  This indicates unbalanced dataset.

concatrev = train.groupby(train['fullVisitorId']).sum()
concatrev = concatrev['totals.transactionRevenue']
mask = concatrev[concatrev > 0]
1-(len(mask))/len(concatrev)


0.9860032737440962

In [8]:
#What is breakdown of visit number?
train['visitNumber'].describe()

#mostly people visit the site once but outliers driving this.  Could we create column for "power user?"


count    903653.000000
mean          2.264897
std           9.283735
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         395.000000
Name: visitNumber, dtype: float64

In [9]:
#some of these columns look like data isn't available.  Let's find out which columns only have 1 value

print(len(train['device.language'].unique()))
for x in train.columns:
    print(x, len(train[x].unique()))

#It looks like a number of columns only have one unique value.  Let's remove these from the dataset.



1
channelGrouping 8
date 366
fullVisitorId 714167
sessionId 902755
socialEngagementType 1
visitId 886303
visitNumber 384
visitStartTime 887159
device.browser 54
device.browserSize 1
device.browserVersion 1
device.deviceCategory 3
device.flashVersion 1
device.isMobile 2
device.language 1
device.mobileDeviceBranding 1
device.mobileDeviceInfo 1
device.mobileDeviceMarketingName 1
device.mobileDeviceModel 1
device.mobileInputSelector 1
device.operatingSystem 20
device.operatingSystemVersion 1
device.screenColors 1
device.screenResolution 1
geoNetwork.city 649
geoNetwork.cityId 1
geoNetwork.continent 6
geoNetwork.country 222
geoNetwork.latitude 1
geoNetwork.longitude 1
geoNetwork.metro 94
geoNetwork.networkDomain 28064
geoNetwork.networkLocation 1
geoNetwork.region 376
geoNetwork.subContinent 23
totals.bounces 2
totals.hits 274
totals.newVisits 2
totals.pageviews 214
totals.transactionRevenue 5333
totals.visits 1
trafficSource.adContent 45
trafficSource.adwordsClickInfo.adNetworkType 3
traff

In [31]:
#only keeping columns that have multiple unique values

colskeep = []
for x in train.columns:
    if len(train[x].unique()) != 1:
        colskeep.append(x)

#going to remove columns that not in test set so df have same cols.  Target variable already saved.
colskeep.remove('totals.transactionRevenue')
colskeep.remove('trafficSource.campaignCode')
trainnew = train[colskeep]
testnew = test[colskeep]


trainnew.shape
        

(903653, 34)

In [32]:
#null values in df?  #mostly not, but where there are nulls, high percentage of columns.  Will figure out how to address later.
trainnew.isna().sum()/len(trainnew)

channelGrouping                                 0.000000
date                                            0.000000
fullVisitorId                                   0.000000
sessionId                                       0.000000
visitId                                         0.000000
visitNumber                                     0.000000
visitStartTime                                  0.000000
device.browser                                  0.000000
device.deviceCategory                           0.000000
device.isMobile                                 0.000000
device.operatingSystem                          0.000000
geoNetwork.city                                 0.000000
geoNetwork.continent                            0.000000
geoNetwork.country                              0.000000
geoNetwork.metro                                0.000000
geoNetwork.networkDomain                        0.000000
geoNetwork.region                               0.000000
geoNetwork.subContinent        

In [33]:
train['trafficSource.adwordsClickInfo.page'].describe()

count    21460.000000
mean         1.008108
std          0.173584
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         14.000000
Name: trafficSource.adwordsClickInfo.page, dtype: float64

In [34]:
# correlation of numeric variables, anything correlate with revenue?

train.corr()

#visit number, totals.hits, and totals.pageviews strongest corr with rev
#hits and pageviews strongly correlated, may want to remove one for ols

,visitId,visitNumber,visitStartTime,device.isMobile,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adwordsClickInfo.page
visitId,1.000000,0.002069,1.000000,0.144332,NaN,-0.027701,NaN,-0.022259,-0.003447,NaN,-0.011634
visitNumber,0.002069,1.000000,0.002069,-0.037667,NaN,0.041317,NaN,0.043363,0.308895,NaN,-0.005168
visitStartTime,1.000000,0.002069,1.000000,0.144332,NaN,-0.027701,NaN,-0.022259,-0.003447,NaN,-0.011634
device.isMobile,0.144332,-0.037667,0.144332,1.000000,NaN,-0.030367,NaN,-0.030199,-0.052623,NaN,0.010172
totals.bounces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals.hits,-0.027701,0.041317,-0.027701,-0.030367,NaN,1.000000,NaN,0.983205,0.141973,NaN,-0.011738
totals.newVisits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals.pageviews,-0.022259,0.043363,-0.022259,-0.030199,NaN,0.983205,NaN,1.000000,0.128644,NaN,-0.012944
totals.transactionRevenue,-0.003447,0.308895,-0.003447,-0.052623,NaN,0.141973,NaN,0.128644,1.000000,NaN,NaN
totals.visits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#add column for month

trainnew['month'] = trainnew['date'].apply(lambda s:s[4:6])
trainnew['month'].describe()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count     903653
unique        12
top           11
freq      113972
Name: month, dtype: object

In [36]:
#histogram of dates

#trainnew['month'].hist(bins=14)

#plt.hist(trainnew['month'], bins=12)
#plt.ylabel('No of times')
#plt.show()



In [37]:
#lets fill in missing values to do random forest
#remove additional columns that are categorical but lots of vars, so would be hard to compute


trainnew.drop(['trafficSource.adwordsClickInfo.gclId','geoNetwork.networkDomain' ], axis = 1, inplace = True)
testnew.drop(['trafficSource.adwordsClickInfo.gclId','geoNetwork.networkDomain' ], axis = 1, inplace = True)

trainnew.shape


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(903653, 33)

In [38]:
naobjects = ['trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.isVideoAd',
'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign','trafficSource.isTrueDirect', 'trafficSource.keyword',
'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source']

for x in naobjects:
    trainnew[x] = trainnew[x].fillna('None')
    testnew[x] = testnew[x].fillna('None')

zero = ['totals.bounces','trafficSource.adwordsClickInfo.page','totals.newVisits', 'totals.pageviews']    
for x in zero:
    trainnew[x] = trainnew[x].fillna(0)
    testnew[x] = testnew[x].fillna(0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [39]:
trainnew.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.region', 'geoNetwork.subContinent',
       'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source', 'month'],
      dtype='object')

In [40]:
trainnew.drop(['fullVisitorId', 'sessionId', 'visitId', 'visitStartTime', 'geoNetwork.country', 'geoNetwork.city', 'trafficSource.referralPath', 'trafficSource.keyword'], axis = 1, inplace = True)
testnew.drop(['fullVisitorId', 'sessionId', 'visitId', 'visitStartTime', 'geoNetwork.country', 'geoNetwork.city', 'trafficSource.referralPath', 'trafficSource.keyword'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [41]:
concat = pd.concat((trainnew,testnew))
concat = pd.get_dummies(concat)
trainexpanded = concat[:len(train)]
testexpanded = concat[len(train):]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [42]:
trainexpanded.shape

(903653, 2087)

In [ ]:
trainexpanded

In [51]:
#fill rev na
#trainhead = trainexpanded.head(100000)
#testhead = testexpanded.head(100000)
#yhead = revenue.head(100000)

revenue = revenue.fillna(0)

model = RandomForestRegressor()
model.fit(trainexpanded, revenue)
predictions = model.predict(testexpanded)

In [52]:
predictions[:100]

array([      0.,       0.,       0.,       0.,       0.,       0.,
       1699000.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,      

In [53]:
predictions = pd.DataFrame(predictions)


In [54]:
ids = test["fullVisitorId"]
predictions['fullVisitorId'] = ids

In [55]:
preds = predictions[0].groupby(predictions['fullVisitorId']).sum()

In [56]:
preds = pd.DataFrame(preds)

In [57]:
preds = preds.reset_index()

In [58]:
preds.columns = ['fullVisitorId', 'PredictedLogRevenue']

In [59]:
preds['PredictedLogRevenue'] = preds['PredictedLogRevenue'].apply(np.log1p)

In [60]:
preds['PredictedLogRevenue'] = preds['PredictedLogRevenue'].fillna(0)

In [63]:
preds.to_csv('predictions4.csv', index = False)

In [ ]:
#aggregate before making predictions
features = trainexpanded.columns

testcombined = testexpanded
traincombined = trainexpanded
trainexpanded['fullVisitorId'] = train['fullVisitorId']
testexpanded['fullVisitorId'] = test['fullVisitorId']
traincombined = trainexpanded[features].groupby('fullVisitorId').mean()
testcombined = trainexpanded[features].groupby('fullVisitorId').mean()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
